# This notebook is dedicated to the data preprocessing of the Research Project
### MSC/DSA/134

In [59]:
# loading required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from globals.data_visualizations import *
from globals.pandas_functions import *
import joblib
from imblearn.over_sampling import SMOTE

In [2]:
# define target column
target_variable = "isFraud"

In [3]:
# define datasets base path
data_base_path = "../data/"

In [34]:
# define global file name prefix
file_name_prefix = "unified_transaction_data_option1"

In [36]:
# import dataset
transaction_data_df = pd.read_csv(data_base_path + f"processed/null_value_option_1/{file_name_prefix}.csv")

In [5]:
dataset_dimension("Transaction Data", transaction_data_df)

Transaction Data dataset dimension: (590509, 27)


## Categorical columns encoding

In [38]:
categorical_columns = get_categorical_columns(transaction_data_df)
print ("Categorical Columns:")
print (categorical_columns)

Categorical Columns:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M6']


In [39]:
show_dataframe_head(transaction_data_df, columns=categorical_columns, n=10)

  ProductCD       card4   card6  P_emaildomain M1 M2 M3  M4 M6
0         W    discover  credit      gmail.com  T  T  T  M2  T
1         W  mastercard  credit      gmail.com  T  T  T  M0  T
2         W        visa   debit    outlook.com  T  T  T  M0  F
3         W  mastercard   debit      yahoo.com  T  T  T  M0  F
4         H  mastercard  credit      gmail.com  T  T  T  M0  F
5         W        visa   debit      gmail.com  T  T  T  M1  T
6         W        visa   debit      yahoo.com  T  T  T  M0  F
7         W        visa   debit       mail.com  T  T  T  M0  F
8         H        visa   debit  anonymous.com  T  T  T  M0  F
9         W  mastercard   debit      yahoo.com  T  T  T  M0  T


,ProductCD,card4,card6,P_emaildomain,M1,M2,M3,M4,M6
0,W,discover,credit,gmail.com,T,T,T,M2,T
1,W,mastercard,credit,gmail.com,T,T,T,M0,T
2,W,visa,debit,outlook.com,T,T,T,M0,F
3,W,mastercard,debit,yahoo.com,T,T,T,M0,F
4,H,mastercard,credit,gmail.com,T,T,T,M0,F
5,W,visa,debit,gmail.com,T,T,T,M1,T
6,W,visa,debit,yahoo.com,T,T,T,M0,F
7,W,visa,debit,mail.com,T,T,T,M0,F
8,H,visa,debit,anonymous.com,T,T,T,M0,F
9,W,mastercard,debit,yahoo.com,T,T,T,M0,T


In [ ]:
# view each categorical column distribution
for col in categorical_columns:
    plot_categorical_distribution(transaction_data_df, col)


### Note:
Encode categorical variables with high cardinality using **target encoding**, and lowerer (less than 2 values) cardinality using **one-hot encoding**

In [41]:
# get cardinality of categorical columns
categorical_cardinality = get_categorical_value_counts(transaction_data_df, categorical_columns)
print ("Categorical Columns Cardinality:")
print (categorical_cardinality)

Categorical Columns Cardinality:
          column  value_count
0      ProductCD            5
1          card4            4
2          card6            4
3  P_emaildomain           59
4             M1            2
5             M2            2
6             M3            2
7             M4            3
8             M6            2


In [42]:
# export this cardinality information to csv for future reference
export_dataframe_to_csv(categorical_cardinality, data_base_path + "processed/null_value_option_1","unified_transaction_data_option1_categorical_cardinality.csv")

DataFrame successfully exported to: data/processed/null_value_option_1\unified_transaction_data_option1_categorical_cardinality.csv


'data/processed/null_value_option_1\\unified_transaction_data_option1_categorical_cardinality.csv'

In [43]:
# one-hot encode categorical columns with less than 2 unique values
max_cardinality_one_hot = 2
columns_for_one_hot = categorical_cardinality.loc[categorical_cardinality["value_count"] <= max_cardinality_one_hot]["column"].tolist()
print(f"Columns to be one-hot encoded (cardinality <= {max_cardinality_one_hot}):")
print(columns_for_one_hot)

Columns to be one-hot encoded (cardinality <= 2):
['M1', 'M2', 'M3', 'M6']


In [44]:
# one-hot encode selected columns
one_hot_encoded_data = one_hot_encode(transaction_data_df, columns_for_one_hot)

In [45]:
one_hot_encoded_data.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card4,card6,P_emaildomain,C1,C2,...,C10,C11,C12,C13,C14,M4,M1_T,M2_T,M3_T,M6_T
0,0,86400,68.5,W,13926,discover,credit,gmail.com,1.0,1.0,...,0.0,2.0,0.0,1.0,1.0,M2,1.0,1.0,1.0,1.0
1,0,86401,29.0,W,2755,mastercard,credit,gmail.com,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,M0,1.0,1.0,1.0,1.0
2,0,86469,59.0,W,4663,visa,debit,outlook.com,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,M0,1.0,1.0,1.0,0.0
3,0,86499,50.0,W,18132,mastercard,debit,yahoo.com,2.0,5.0,...,0.0,1.0,0.0,25.0,1.0,M0,1.0,1.0,1.0,0.0
4,0,86506,50.0,H,4497,mastercard,credit,gmail.com,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,M0,1.0,1.0,1.0,0.0


### Note:
The remaining categorical columns with high cardinality will be target encoded after splitting the dataset into train and test sets to avoid data leakage

In [46]:
# split dataset into train and test sets
target_df = one_hot_encoded_data[target_variable]
one_hot_encoded_data = drop_columns(one_hot_encoded_data, [target_variable])
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(one_hot_encoded_data, target_df, test_size=0.2, random_state=42, stratify=target_df)
# reset index
X_train_df.reset_index(drop=True, inplace=True)
X_test_df.reset_index(drop=True, inplace=True)
y_train_df.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

# show dimensions of train and test sets
dataset_dimension("X Train Set", X_train_df)
dataset_dimension("X Test Set", X_test_df)
dataset_dimension("y Train Set", y_train_df)
dataset_dimension("y Test Set", y_test_df)


X Train Set dataset dimension: (472407, 26)
X Test Set dataset dimension: (118102, 26)
y Train Set dataset dimension: (472407,)
y Test Set dataset dimension: (118102,)


In [16]:
# get remaining categorical columns for target encoding
categorical_cols_for_target_encoding = [col for col in categorical_columns if col not in columns_for_one_hot]
print("Categorical Columns for Target Encoding:")
print(categorical_cols_for_target_encoding)

Categorical Columns for Target Encoding:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'M4']


In [47]:
# target encoding for remaining categorical columns
target_encoder = TargetEncoder(cols=categorical_cols_for_target_encoding, smoothing=5)

X_train_encoded_df = target_encoder.fit_transform(X_train_df, y_train_df)
X_test_encoded_df = target_encoder.transform(X_test_df)

In [48]:
dataset_dimension("X Train Encoded Set", X_train_encoded_df)
dataset_dimension("X Test Encoded Set", X_test_encoded_df)

X Train Encoded Set dataset dimension: (472407, 26)
X Test Encoded Set dataset dimension: (118102, 26)


In [57]:
# export target encoder object for future use
fitted_modules_base = "fitted_modules/"
encoder_export_path = fitted_modules_base + "null_value_option_1/target_encoder.joblib"
joblib.dump(target_encoder, encoder_export_path)

['fitted_modules/null_value_option_1/target_encoder.joblib']

In [49]:
X_train_encoded_df.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card4,card6,P_emaildomain,C1,C2,C3,...,C10,C11,C12,C13,C14,M4,M1_T,M2_T,M3_T,M6_T
0,4888320,87.95,0.020425,12544,0.034471,0.024239,0.039408,1.0,2.0,0.0,...,0.0,1.0,0.0,21.0,1.0,0.026075,1.0,0.0,0.0,0.0
1,568144,92.00,0.020425,7919,0.034671,0.024239,0.039408,5.0,5.0,0.0,...,0.0,2.0,0.0,3.0,3.0,0.026075,1.0,1.0,1.0,0.0
2,2574513,92.00,0.020425,12686,0.034471,0.024239,0.039408,1.0,1.0,0.0,...,0.0,2.0,0.0,8.0,1.0,0.026075,1.0,1.0,1.0,1.0
3,12256532,150.00,0.047466,18227,0.034471,0.066792,0.039408,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.026075,1.0,1.0,1.0,0.0
4,12065455,117.00,0.020425,3181,0.034471,0.024239,0.039408,5.0,4.0,0.0,...,0.0,3.0,0.0,20.0,4.0,0.026075,1.0,1.0,1.0,0.0


In [53]:
# export preprocessed datasets to csv
export_dir = data_base_path + "processed/null_value_option_1"
export_dataframe_to_csv(X_train_encoded_df, export_dir,"unified_transaction_data_option1_x_train.csv")
export_dataframe_to_csv(X_test_encoded_df, export_dir,"unified_transaction_data_option1_x_test.csv")
export_dataframe_to_csv(y_train_df, export_dir,"unified_transaction_data_option1_y_train.csv")
export_dataframe_to_csv(y_test_df, export_dir,"unified_transaction_data_option1_y_test.csv")

DataFrame successfully exported to: data/processed/null_value_option_1\unified_transaction_data_option1_x_train.csv
DataFrame successfully exported to: data/processed/null_value_option_1\unified_transaction_data_option1_x_test.csv
DataFrame successfully exported to: data/processed/null_value_option_1\unified_transaction_data_option1_y_train.csv
DataFrame successfully exported to: data/processed/null_value_option_1\unified_transaction_data_option1_y_test.csv


'data/processed/null_value_option_1\\unified_transaction_data_option1_y_test.csv'

### Normalization/Standardization

In [54]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded_df)
X_test_scaled = scaler.transform(X_test_encoded_df)

dataset_dimension("X Train Scaled Set", X_train_scaled)
dataset_dimension("X Test Scaled Set", X_test_scaled)

X Train Scaled Set dataset dimension: (472407, 26)
X Test Scaled Set dataset dimension: (118102, 26)


In [58]:
scaler_export_path = fitted_modules_base + "null_value_option_1/standard_scaler.joblib"
joblib.dump(scaler, scaler_export_path)

['fitted_modules/null_value_option_1/standard_scaler.joblib']

In [55]:
X_train_scaled

array([[-0.53824074, -0.19538213, -0.47244596, ..., -4.05681655,
        -2.78293283, -0.69729496],
       [-1.47403698, -0.17856975, -0.47244596, ...,  0.2464987 ,
         0.35933314, -0.69729496],
       [-1.03943603, -0.17856975, -0.47244596, ...,  0.2464987 ,
         0.35933314,  1.43411333],
       ...,
       [-1.14679305, -0.35292027,  0.09006689, ...,  0.2464987 ,
         0.35933314, -0.69729496],
       [-1.37729042, -0.35707148, -0.47244596, ...,  0.2464987 ,
         0.35933314, -0.69729496],
       [ 1.77218591,  0.27370379, -0.47244596, ...,  0.2464987 ,
         0.35933314,  1.43411333]])

### Class Imbalance Handling
Balancing only the training set using SMOTE

In [60]:
# TODO: complete SMOTE implementation
smote = SMOTE(random_state=42, sampling_strategy='auto')

X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train_df)

print("Original data shape")
dataset_dimension("X Train scaled Set", X_train_scaled)
dataset_dimension("Y Train set", y_train_df)

print("\nBalanced data shape")
dataset_dimension("X Train Balanced Set", X_train_balanced)
dataset_dimension("y Train Balanced Set", y_train_balanced)

Original data shape
X Train scaled Set dataset dimension: (472407, 26)
Y Train set dataset dimension: (472407,)
Balanced data shape
X Train Balanced Set dataset dimension: (911764, 26)
y Train Balanced Set dataset dimension: (911764,)


In [61]:
# export balanced training set to csv
export_dir = data_base_path + "processed/null_value_option_1/scaled_and_balanced"
export_dataframe_to_csv(pd.DataFrame(X_train_balanced), export_dir,"unified_transaction_data_option1_x_train_balanced.csv")
export_dataframe_to_csv(pd.DataFrame(y_train_balanced), export_dir,"unified_transaction_data_option1_y_train_balanced.csv")

export_dataframe_to_csv(pd.DataFrame(X_test_scaled), export_dir,"unified_transaction_data_option1_x_test_scaled.csv")
export_dataframe_to_csv(y_test_df, export_dir,"unified_transaction_data_option1_y_test.csv")

DataFrame successfully exported to: data/processed/null_value_option_1/scaled_and_balanced\unified_transaction_data_option1_x_train_balanced.csv
DataFrame successfully exported to: data/processed/null_value_option_1/scaled_and_balanced\unified_transaction_data_option1_y_train_balanced.csv
DataFrame successfully exported to: data/processed/null_value_option_1/scaled_and_balanced\unified_transaction_data_option1_x_test_scaled.csv
DataFrame successfully exported to: data/processed/null_value_option_1/scaled_and_balanced\unified_transaction_data_option1_y_test.csv


'data/processed/null_value_option_1/scaled_and_balanced\\unified_transaction_data_option1_y_test.csv'